In [15]:
with open("/Users/hkmac/Downloads/input.txt") as f:
    inp = f.read().strip().split("\n")


In [16]:
inp = [[int(y) for y in list(x)] for x in inp]
# inp = [(x,int(y)) for x,y in inp]

In [33]:
from collections import Counter
pos = Counter()
for row in inp:
    for i,x in enumerate(row):
        pos[11-i] += x

In [34]:
len(inp)

1000

In [35]:
res = 0
for i,v in pos.items():
    if v > 500:
        res += 1 * 2**i
        
r2 = 0
for i,v in pos.items():
    if v < 500:
        r2 += 1 * 2**i

In [36]:
pos

Counter({11: 489,
         10: 493,
         9: 504,
         8: 517,
         7: 513,
         6: 485,
         5: 521,
         4: 479,
         3: 502,
         2: 506,
         1: 480,
         0: 507})

In [37]:
res * r2

2967914

In [28]:
res

2908

In [5]:
# inp

In [14]:
x,y,aim = 0,0,0

for a,b in inp:
    if a == "up":
        aim -= b
    if a == "down":
        aim += b
    if a == "forward":
        x += b
        y += aim*b

In [15]:
x,y

(1940, 1007368)

In [16]:
x*y


1954293920